# Model Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

In [3]:
PATH_TO_SAVE_DATA = "."
PATH_DF_POS_FR = PATH_TO_SAVE_DATA + '/' + 'df_pos_fr.csv'
PATH_DF_TEST_FR = PATH_TO_SAVE_DATA + '/' + 'df_test_fr.csv'
PATH_JSON_METEO_FR = PATH_TO_SAVE_DATA + '/' + 'data_meteo_fr.json'
PATH_DF_FEAT_FR = PATH_TO_SAVE_DATA + '/' + 'df_feat_fr.csv' 
PATH_GEO_DEP_FR = PATH_TO_SAVE_DATA + '/sources/geofrance/' + 'departments.csv'
PATH_MDL_SINGLE_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_single_step_pos_fr"
PATH_MDL_MULTI_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_multi_step_pos_fr"
PATH_MDL_MULTI_TFLITE = PATH_TO_SAVE_DATA + '/' + \
    'serverless/tensorflow-lite-on-aws-lambda'
PATH_MDL_MULTI_TFLITE_FILE = PATH_MDL_MULTI_TFLITE + '/' + \
    "converted_model.tflite"
PATH_SERVERLESS = PATH_MDL_MULTI_TFLITE + '/' + 'serverless.yml'

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140734 # on 13/05/2020
NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer'

# model 
PAST_HISTORY = 14 # days used to predict next values in future
FUTURE_TARGET = 7 # nb predict days later
STEP = 1

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")

TRAIN_SPLIT = 135


### load data

In [4]:
df_feat_fr = pd.read_csv(PATH_DF_FEAT_FR)
df_feat_fr.index = df_feat_fr["date"]
df_feat_fr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_fr.shape[0])]
df_feat_fr


,T_min,date,T_max,H_min,H_max,pos,age_pos,test,age_test,day_num,nb_cases,train
date,,,,,,,,,,,,
2020-05-13,284.926667,2020-05-13,290.505000,64.661017,88.135593,882,60.987528,38787,55.444840,3,141109,True
2020-05-14,285.050000,2020-05-14,290.963333,59.406780,84.847458,980,60.435714,41565,54.798003,4,142089,True
2020-05-15,285.308333,2020-05-15,291.920000,57.372881,82.966102,1031,59.838991,47019,54.313320,5,143120,True
2020-05-16,284.956667,2020-05-16,293.500000,53.741379,86.534483,291,60.158076,16145,54.355528,6,143411,True
2020-05-17,285.598333,2020-05-17,294.446667,49.879310,85.500000,139,61.568345,6270,58.062360,0,143550,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-05,285.878333,2020-10-05,290.990000,67.779661,91.186441,20947,44.384112,178475,46.055268,1,645995,False
2020-10-06,286.891667,2020-10-06,292.201667,67.508475,89.983051,19020,44.494742,161078,44.712059,2,665015,False
2020-10-07,286.887288,2020-10-07,292.333051,62.500000,90.655172,20032,43.769020,160191,43.757926,3,685047,False


### prepare features

In [5]:
features = df_feat_fr.copy().filter(items=['T_min', 'T_max', 'H_min',
                                           'H_max', 'pos', 'test', 'day_num',
                                          'age_pos', 'age_test'])
features

,T_min,T_max,H_min,H_max,pos,test,day_num,age_pos,age_test
date,,,,,,,,,
2020-05-13,284.926667,290.505000,64.661017,88.135593,882,38787,3,60.987528,55.444840
2020-05-14,285.050000,290.963333,59.406780,84.847458,980,41565,4,60.435714,54.798003
2020-05-15,285.308333,291.920000,57.372881,82.966102,1031,47019,5,59.838991,54.313320
2020-05-16,284.956667,293.500000,53.741379,86.534483,291,16145,6,60.158076,54.355528
2020-05-17,285.598333,294.446667,49.879310,85.500000,139,6270,0,61.568345,58.062360
...,...,...,...,...,...,...,...,...,...
2020-10-05,285.878333,290.990000,67.779661,91.186441,20947,178475,1,44.384112,46.055268
2020-10-06,286.891667,292.201667,67.508475,89.983051,19020,161078,2,44.494742,44.712059
2020-10-07,286.887288,292.333051,62.500000,90.655172,20032,160191,3,43.769020,43.757926


### Prepare dataset

In [6]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
dataset = (dataset-data_mean)/data_std

In [7]:
dataset.shape[1]

9

In [8]:
PAST_HISTORY

14

In [9]:
dataset.shape[1]

9

### Load model

In [10]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x64022a3d0> and <tensorflow.python.keras.layers.core.Dropout object at 0x6402a1350>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x6402a1bd0> and <tensorflow.python.keras.layers.core.Dense object at 0x6402b51d0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x6402a1bd0> and <tensorflow.python.keras.layers.core.Dropout object at 0x6402a1350>).
CPU times: user 2.11 s, sys: 152 ms, total: 2.26 s
Wall time: 2.68 s


In [11]:
multi_step_model.inputs[0].dtype

tf.float32

In [12]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_lstm"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./keras_lstm/assets


### Save model TFlite

In [18]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE)

File ./serverless/tensorflow-lite-on-aws-lambda/converted_model_20201013_16_10_52.tflite moved!


In [19]:
open(PATH_MDL_MULTI_TFLITE_FILE, "wb").write(tflite_model)

6104

## Test converted model

### Predict with TFlite & Compare 

In [13]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[58 - 72]
[65 - 79]
[72 - 86]
[79 - 93]
[86 - 100]
[93 - 107]
[100 - 114]
[107 - 121]
[114 - 128]
9


array([[  794.53906,   573.5635 ,   737.1929 ,   900.4518 ,  1174.9934 ,
         1443.3275 ,  1514.0929 ,  1121.9463 ,   728.1682 ,  1069.7264 ,
         1386.3137 ,  1821.1062 ,  2134.7217 ,  2111.8508 ,  1611.0874 ,
         1343.4803 ,  1666.6522 ,  1936.792  ,  2481.4824 ,  2893.221  ,
         2925.7922 ,  2851.1912 ,  2217.1235 ,  2752.3496 ,  3274.1917 ,
         4321.4863 ,  4639.918  ,  4477.752  ,  4013.4312 ,  2749.5015 ,
         3690.4539 ,  4731.7075 ,  6523.2383 ,  6793.405  ,  6274.082  ,
         6107.8823 ,  3913.3142 ,  4726.2705 ,  6161.127  ,  9262.326  ,
         9705.555  ,  8793.442  ,  8174.411  ,  4993.5337 ,  5670.9854 ,
         7451.4326 , 11373.937  , 11368.053  , 10189.23   ,  9129.875  ,
         5029.9014 ,  6804.997  ,  9114.3    , 12794.654  , 12615.478  ,
        11984.076  , 10026.873  ,  5788.4688 ,  7669.9277 , 10158.219  ,
        13777.826  , 12801.566  , 11832.221  ]], dtype=float32)

In [14]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [15]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE)

In [16]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow-lite-on-aws-lambda/converted_model.tflite'

### Predict reloaded model

In [17]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [18]:
x_multi.shape

(1, 14, 9)

In [19]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

In [20]:
dataset.shape

(150, 9)

In [21]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[58 - 72]
[65 - 79]
[72 - 86]
[79 - 93]
[86 - 100]
[93 - 107]
[100 - 114]
[107 - 121]
[114 - 128]


In [22]:
# patch for simulate import in AWS
import tensorflow.lite as tflite

# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
def predict(event, context):
    # retrieve entry event -> json_list_list_x
    json_list_list_x = event.get('body')
    print(json_list_list_x[0:50])
    list_list_in = json.loads(json.loads(json_list_list_x))
    # convert in list of array
    # retrieve json into lambda
    list_arr_in = []
    for list_curr in list_list_in:
        list_arr_in.append(np.array(list_curr))

    print("INPUT : nb. arrays : {} / arrays shape: {}".format(len(list_arr_in), 
                                             list_arr_in[0].shape))

    # prepare TFlite model
    interpreter = tflite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()


    # Run the model with TensorFlow Lite
    list_list_out = []
    for x_multi in list_arr_in:
        interpreter.set_tensor(input_details[0]["index"], 
                               x_multi.astype(np.float32))
        interpreter.invoke()
        result = interpreter.get_tensor(output_details[0]["index"])
        list_list_out.append(result.tolist())
        # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
        # the states.
        # Clean up internal states.
        interpreter.reset_all_variables()
    print("OUTPUT : nb. arrays : {} / arrays shape in list: {}" \
          .format(len(list_list_out), np.array(list_list_out[0]).shape))

    # Prepare output
    json_list_list_out = json.dumps(list_list_out)
    response = {
        "statusCode": 200,
        "body": json_list_list_out
    }
    return response

context = None
response = predict(event, context)


"[[[[0.8661074471642011, -0.02564300546570835, 0.3
INPUT : nb. arrays : 9 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [23]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [24]:
y_multi_pred

array([[-0.5215902 , -0.5768285 , -0.53592527, -0.49511465, -0.4264862 ,
        -0.35940948, -0.34171993, -0.43974665, -0.5381812 , -0.45280027,
        -0.3736615 , -0.2649744 , -0.18657842, -0.19229557, -0.31747374,
        -0.38436875, -0.30358395, -0.23605584, -0.09989706,  0.00302715,
         0.01116916, -0.00747924, -0.16598004, -0.0321871 ,  0.09826019,
         0.36005735,  0.4396572 ,  0.3991198 ,  0.28305134, -0.03289908,
         0.20231518,  0.4626022 ,  0.91043955,  0.97797436,  0.8481568 ,
         0.80661106,  0.25802463,  0.46124315,  0.81992096,  1.5951422 ,
         1.705938  ,  1.4779332 ,  1.3231909 ,  0.52805215,  0.69739795,
         1.1424645 ,  2.122991  ,  2.1215203 ,  1.8268447 ,  1.5620328 ,
         0.53714323,  0.9808721 ,  1.5581393 ,  2.4781346 ,  2.433345  ,
         2.2755105 ,  1.7862595 ,  0.7267658 ,  1.1970828 ,  1.8190925 ,
         2.7239027 ,  2.4798625 ,  2.2375507 ]], dtype=float32)

In [25]:
y_multi_pred_out

array([[-0.52159017, -0.57682848, -0.53592527, -0.49511465, -0.42648619,
        -0.35940948, -0.34171993, -0.43974674, -0.53818125, -0.45280027,
        -0.37366149, -0.26497436, -0.18657845, -0.19229554, -0.31747374,
        -0.38436875, -0.30358392, -0.23605584, -0.09989709,  0.00302715,
         0.01116917, -0.00747922, -0.16598004, -0.03218707,  0.09826022,
         0.36005738,  0.43965727,  0.39911985,  0.28305134, -0.0328991 ,
         0.20231497,  0.46260199,  0.91043949,  0.9779743 ,  0.84815669,
         0.806611  ,  0.2580246 ,  0.46124333,  0.81992114,  1.59514225,
         1.70593798,  1.47793329,  1.32319105,  0.52805221,  0.69739789,
         1.1424644 ,  2.12299085,  2.12152004,  1.82684469,  1.56203282,
         0.53714323,  0.98087209,  1.55813932,  2.47813416,  2.4333446 ,
         2.27551055,  1.78625953,  0.72676569,  1.19708264,  1.81909239,
         2.7239027 ,  2.47986221,  2.2375505 ]])

In [26]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [27]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

"[[[[-2.324386895944945, -2.639625843078335, 0.768
INPUT : nb. arrays : 1 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [28]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

In [29]:
y_future_pred_out

array([[1.25671518, 0.26149568, 0.71327579, 1.26555955, 2.33228302,
        2.35756588, 1.97211969]])

In [30]:
y_future_pred

array([[1.2567154, 0.2614957, 0.7132758, 1.2655597, 2.3322833, 2.3575656,
        1.9721197]], dtype=float32)

In [31]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda
    
- Execute : sls deploy -v

In [33]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd ./serverless/tensorflow-lite-on-aws-lambda\nserverless deploy -v'

In [34]:
open('deploy_serverless.sh', "w").write(str_exe)
os.chmod('deploy_serverless.sh', stat.S_IRWXU)

In [35]:
!cat ./deploy_serverless.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd ./serverless/tensorflow-lite-on-aws-lambda
serverless deploy -v

In [36]:
!./deploy_serverless.sh

Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow-lite-on-aws-lambda/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow-lite-on-aws-lambda/.serverless/requirements.txt...
Serverless: Using static cache of requirements found at /Users/gregory/Library/Caches/serverless-python-requirements/ef4e42eb03bbad46f74fee99a3c01f994e8119e7a557f947779093ab37b248d3_slspyc ...
Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: Injecting required Python packages to package...
Serverless: Uploading CloudFormation file to S3...
Serverless: Uploading artifacts...
Serverless: Uploading service tensorflow-lite-on-aws-lambda.zip file to S3 (18.4 MB)...
Serverless: Validating template...
Serverless: Updating Stack...
Serverless: Checking Stack update progress...
CloudF

### API AWS real Test

In [37]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[58 - 72]
[65 - 79]
[72 - 86]
[79 - 93]
[86 - 100]
[93 - 107]
[100 - 114]
[107 - 121]
[114 - 128]
status code :  200
[[[-0.5216001272201538, -0.5768371820449829, -0.5359305739402771, -0.4951191246509552, -0.42649587988853455, -0.35942143201828003, -0.34173017740249634]], [[-0.43972229957580566, -0.5381525158882141, -0.4527134299278259, -0.3735485374927521, -0.2648811340332031, -0.18651607632637024, -0.19224025309085846]], [[-0.3174644708633423, -0.38433074951171875, -0.30356526374816895, -0.23604823648929596, -0.09988704323768616, 0.00302993506193161, 0.011154964566230774]], [[-0.007503926753997803, -0.16600263118743896, -0.03220542520284653, 0.09823962301015854, 0.3600359559059143, 0.4396497905254364, 0.39911484718322754]], [[0.2830981910228729, -0.03286878764629364, 0.20234274864196777, 0.4626409113407135, 0.910483717918396, 0.9779927134513855, 0.8481712937355042]], [[0.80661541223526, 0.25803759694099426, 0.4612640142440796, 0.8199437856674194, 1.5951529741287231, 1.7059340476989746

In [38]:
len(json_list_list_x)

23745

In [39]:
resp.json()

[[[-0.5216001272201538,
   -0.5768371820449829,
   -0.5359305739402771,
   -0.4951191246509552,
   -0.42649587988853455,
   -0.35942143201828003,
   -0.34173017740249634]],
 [[-0.43972229957580566,
   -0.5381525158882141,
   -0.4527134299278259,
   -0.3735485374927521,
   -0.2648811340332031,
   -0.18651607632637024,
   -0.19224025309085846]],
 [[-0.3174644708633423,
   -0.38433074951171875,
   -0.30356526374816895,
   -0.23604823648929596,
   -0.09988704323768616,
   0.00302993506193161,
   0.011154964566230774]],
 [[-0.007503926753997803,
   -0.16600263118743896,
   -0.03220542520284653,
   0.09823962301015854,
   0.3600359559059143,
   0.4396497905254364,
   0.39911484718322754]],
 [[0.2830981910228729,
   -0.03286878764629364,
   0.20234274864196777,
   0.4626409113407135,
   0.910483717918396,
   0.9779927134513855,
   0.8481712937355042]],
 [[0.80661541223526,
   0.25803759694099426,
   0.4612640142440796,
   0.8199437856674194,
   1.5951529741287231,
   1.7059340476989746,
   1.

In [40]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [41]:
y_multi_pred_out

array([[-0.52160013, -0.57683718, -0.53593057, -0.49511912, -0.42649588,
        -0.35942143, -0.34173018, -0.4397223 , -0.53815252, -0.45271343,
        -0.37354854, -0.26488113, -0.18651608, -0.19224025, -0.31746447,
        -0.38433075, -0.30356526, -0.23604824, -0.09988704,  0.00302994,
         0.01115496, -0.00750393, -0.16600263, -0.03220543,  0.09823962,
         0.36003596,  0.43964979,  0.39911485,  0.28309819, -0.03286879,
         0.20234275,  0.46264091,  0.91048372,  0.97799271,  0.84817129,
         0.80661541,  0.2580376 ,  0.46126401,  0.81994379,  1.59515297,
         1.70593405,  1.47792411,  1.32322538,  0.52806145,  0.6974383 ,
         1.14253223,  2.12304163,  2.12152719,  1.82685339,  1.56202686,
         0.53714067,  0.98083735,  1.5580982 ,  2.47811627,  2.43332601,
         2.27547479,  1.78627455,  0.72676694,  1.19703221,  1.81902885,
         2.72388291,  2.4798615 ,  2.23754287]])

In [42]:
y_multi_pred

array([[-0.52160007, -0.5768372 , -0.5359306 , -0.49511912, -0.42649588,
        -0.35942143, -0.34173024, -0.4397223 , -0.53815246, -0.45271343,
        -0.37354854, -0.26488113, -0.18651608, -0.19224025, -0.31746444,
        -0.38433075, -0.30356523, -0.2360482 , -0.09988704,  0.00302994,
         0.01115497, -0.00750393, -0.1660026 , -0.03220541,  0.09823963,
         0.36003593,  0.4396498 ,  0.39911485,  0.28309825, -0.03286874,
         0.20234275,  0.4626409 ,  0.91048384,  0.9779927 ,  0.8481713 ,
         0.8066154 ,  0.25803754,  0.46126416,  0.81994396,  1.5951532 ,
         1.705934  ,  1.4779243 ,  1.3232255 ,  0.52806145,  0.69743824,
         1.1425322 ,  2.1230419 ,  2.1215272 ,  1.8268535 ,  1.5620267 ,
         0.5371407 ,  0.9808372 ,  1.5580981 ,  2.4781163 ,  2.433326  ,
         2.2754745 ,  1.7862746 ,  0.726767  ,  1.1970323 ,  1.8190287 ,
         2.7238827 ,  2.4798615 ,  2.2375429 ]], dtype=float32)

In [43]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [44]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
# REQUEST URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer' 
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[2.258720874786377, 1.2291589975357056, 0.11853042989969254, 0.07443636655807495, 2.296689748764038, 2.832331895828247, 2.181868076324463]]]


In [45]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[2.25872087, 1.229159  , 0.11853043, 0.07443637, 2.29668975,
        2.8323319 , 2.18186808]])

In [46]:
y_future_pred

array([[2.2587209 , 1.2291588 , 0.11853043, 0.07443652, 2.2966897 ,
        2.8323317 , 2.1818683 ]], dtype=float32)

In [47]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
